In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.losses import BinaryCrossentropy

In [3]:
G_train_df      = pd.read_csv ('data/preprocessed_data/G_train.csv')
T_train_df      = pd.read_csv ('data/preprocessed_data/T_train.csv')
G_T_train_df    = pd.read_csv ('data/preprocessed_data/balanced_G_T_train.csv')

G_cv_df         = pd.read_csv ('data/preprocessed_data/G_cv.csv')
T_cv_df         = pd.read_csv ('data/preprocessed_data/T_cv.csv')
G_T_cv_df        = pd.read_csv ('data/preprocessed_data/G_T_cv.csv')



In [4]:
random_state = 13
frac = 0.15

G_train_df      = G_train_df.sample(frac = frac, random_state=random_state)
T_train_df      = T_train_df.sample(frac = frac, random_state=random_state)
G_T_train_df    = G_T_train_df.sample(frac = frac, random_state=random_state)


In [5]:
G_T_train_df.shape

(19318, 3)

In [6]:
y_train = G_T_train_df.drop(columns = ['group ID','technique ID' ]).values
y_train.dtype

# G_train = sampled_G_train_df.drop(columns = ids)
G_train = G_train_df.drop(columns = 'group ID').values

# T_train = sampled_T_train_df.drop(columns = ids)
T_train = T_train_df.drop(columns = 'technique ID').values

```python
dataset = tf.data.Dataset.from_tensor_slices(({'input_Group': G_train_tf, 'input_Technique': T_train_tf}, y_train_tf))
```

1. `tf.data.Dataset`: This is a class in TensorFlow that represents a potentially large collection of elements. It's used to create pipelines for data processing and input to machine learning models.

2. `from_tensor_slices`: This is a method of the `tf.data.Dataset` class. It is used to create a dataset by slicing the given tensors along their first dimension (usually representing the number of samples). This is a convenient way to create a dataset where each element corresponds to a pair of inputs and their corresponding targets.

3. `({'input_Group': G_train_tf, 'input_Technique': T_train_tf}, y_train_tf)`: This is a tuple containing two elements. The first element is a dictionary that maps the input tensor names (`'input_Group'` and `'input_Technique'`) to their corresponding tensor data (`G_train_tf` and `T_train_tf`). The second element is the target tensor data `y_train_tf`. ❗The  names used in the from_tensor_slices dictionary should match the names of the input layers in your model. This ensures that the data from the dataset is correctly mapped to the corresponding input layers during training.


So, the overall process of this line of code is to create a dataset where each element is a pair of dictionaries (`{'input_Group': ..., 'input_Technique': ...}`) representing the input tensors and their corresponding target tensor.

When you iterate through this dataset, you'll get pairs like:
```
({'input_Group': G_train_sample, 'input_Technique': T_train_sample}, y_train_sample)
```

Here, `G_train_sample` and `T_train_sample` are individual samples from your `input_Group` and `input_Technique` tensors, and `y_train_sample` is the corresponding target value for that sample. This dataset structure is suitable for training machine learning models where you have multiple input features.

In [7]:
G_train_tf = tf.convert_to_tensor(G_train)
T_train_tf = tf.convert_to_tensor(T_train)
y_train_tf = tf.convert_to_tensor(y_train)
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_Group': G_train_tf, 'input_Technique': T_train_tf}, y_train_tf))

batch_size = 32


train_dataset = train_dataset.shuffle(buffer_size=len(G_train_tf))
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [8]:
y_cv = G_T_cv_df.drop(columns = ['group ID','technique ID' ]).values
y_cv.dtype

# G_cv = sampled_G_cv_df.drop(columns = ids)
G_cv = G_cv_df.drop(columns = 'group ID').values

# T_cv = sampled_T_cv_df.drop(columns = ids)
T_cv = T_cv_df.drop(columns = 'technique ID').values

In [9]:
G_cv_tf = tf.convert_to_tensor(G_cv)
T_cv_tf = tf.convert_to_tensor(T_cv)
y_cv_tf = tf.convert_to_tensor(y_cv)
cv_dataset = tf.data.Dataset.from_tensor_slices(({'input_Group': G_cv_tf, 'input_Technique': T_cv_tf}, y_cv_tf))
cv_dataset = cv_dataset.batch(batch_size)

---

In [10]:
# input shapes config
num_G_features = G_train.shape[1]  # remove Group ID during training
num_T_features = T_train.shape[1]   # remove Movie ID during training

# output
num_outputs = 32

tf.random.set_seed(random_state)

# Group NN
Group_NN = tf.keras.models.Sequential(
    layers=[
    tf.keras.layers.Dense (256, activation = 'relu'),
    tf.keras.layers.Dense (128, activation = 'relu'),
    tf.keras.layers.Dense (num_outputs, activation  = 'linear'),
    ], 
    name= "Group_NN")
# input vector for user_NN
input_Group = tf.keras.layers.Input(shape = (num_G_features), name = "input_Group")
vg = Group_NN(input_Group)
# vg = tf.linalg.l2_normalize(vg, axis=1)

# Technique NN
Technique_NN = tf.keras.models.Sequential(
    layers = [
    tf.keras.layers.Dense (256, activation = 'relu'),
    tf.keras.layers.Dense (128, activation = 'relu'),
    tf.keras.layers.Dense (num_outputs, activation  = 'linear'),  
    ],
    name = "Technique_NN")
# input vector for Technique_NN
input_Technique = tf.keras.layers.Input (shape= (num_T_features), name = "input_Technique")
vt = Technique_NN (input_Technique)
# vt = tf.linalg.l2_normalize (vt, axis = 1)

output = tf.keras.layers.Dot (axes=1)(inputs= [vg, vt])



In [11]:
tf.random.set_seed(random_state)
opt = keras.optimizers.Adam (learning_rate= 0.05)
model = tf.keras.Model (inputs = [input_Group, input_Technique],
                        outputs = output, name = 'recsysNN_model')
model.compile (optimizer = opt, loss = BinaryCrossentropy (from_logits= True))


## Early stopping

In [12]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss for early stopping
    patience=5,           # Number of epochs with no improvement before stopping
    restore_best_weights=True  # Restore model weights from the epoch with the best validation loss
)

In [13]:
epochs = 30
history = model.fit(train_dataset,
                    validation_data = cv_dataset,
                    epochs = epochs,
                    callbacks = [early_stopping])


Epoch 1/30


604/604 [==============================] - 5s 6ms/step - loss: 0.8916 - val_loss: 0.5091
Epoch 2/30
604/604 [==============================] - 3s 5ms/step - loss: 0.6242 - val_loss: 0.7446
Epoch 3/30
604/604 [==============================] - 4s 6ms/step - loss: 0.7102 - val_loss: 0.7789
Epoch 4/30
604/604 [==============================] - 4s 6ms/step - loss: 0.6779 - val_loss: 0.7535
Epoch 5/30
604/604 [==============================] - 3s 5ms/step - loss: 0.6738 - val_loss: 0.6759
Epoch 6/30
604/604 [==============================] - 4s 6ms/step - loss: 0.6735 - val_loss: 0.6569


In [14]:
import matplotlib.pyplot as plt

# Access training and validation loss history
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

# Plot the training and validation loss
# plt.plot(range(1, len(training_loss) + 1), training_loss, label='Training Loss')
# plt.plot(range(1, len(validation_loss) + 1), validation_loss, label='Validation Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()


In [15]:
print(len(training_loss))
print(len(validation_loss))

6
6


In [16]:
print (training_loss)
print (validation_loss)

[0.8915860652923584, 0.6242129802703857, 0.7102217674255371, 0.6779444217681885, 0.6737833619117737, 0.6735294461250305]
[0.5090712904930115, 0.7446485757827759, 0.7788713574409485, 0.7535215020179749, 0.6758723258972168, 0.6569448113441467]


In [17]:
# Plot the training and validation loss
# plt.plot(range(1, len(training_loss) + 1), training_loss, label='Training Loss')
# plt.plot(range(1, len(validation_loss) + 1), validation_loss, label='Validation Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

In [18]:
# plt.plot(range(1, len(training_loss) + 1), training_loss, label='Training Loss')